In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("Preprocessor pipeline 1")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")

# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()
# Load the csv file from the chosen dataset into the df variable to preprocess in the rest of the notebook.
df = spark.read.format("csv").option("header", "true") \
       .load("/home/jovyan/data/energy_dataset.csv")

In [2]:
# Renaming all columns in such a way that we do not have spaces in the name for the purpose of further preprocessing.
df2 = df.withColumnRenamed("generation biomass","generation_biomass") \
    .withColumnRenamed("time","ts") \
    .withColumnRenamed("generation fossil brown coal/lignite","generation_fossil_brown_coal_lignite") \
    .withColumnRenamed("generation fossil coal-derived gas","generation_fossil_coal_derived_gas") \
    .withColumnRenamed("generation fossil gas","generation_fossil_gas") \
    .withColumnRenamed("generation fossil hard coal","generation_fossil_hard_coal") \
    .withColumnRenamed("generation fossil oil","generation_fossil_oil") \
    .withColumnRenamed("generation fossil oil shale","generation_fossil_oil_shale") \
    .withColumnRenamed("generation fossil peat","generation_fossil_peat") \
    .withColumnRenamed("generation geothermal","generation_geothermal") \
    .withColumnRenamed("generation hydro pumped storage aggregated","generation_hydro_pumped_storage_aggregated") \
    .withColumnRenamed("generation hydro pumped storage consumption","generation_hydro_pumped_storage_consumption") \
    .withColumnRenamed("generation hydro run-of-river and poundage","generation_hydro_run_of_river_and_poundage") \
    .withColumnRenamed("generation hydro water reservoir","generation_hydro_water_reservoir") \
    .withColumnRenamed("generation marine","generation_marine") \
    .withColumnRenamed("generation nuclear","generation_nuclear") \
    .withColumnRenamed("generation other","generation_other") \
    .withColumnRenamed("generation other renewable","generation_other_renewable") \
    .withColumnRenamed("generation solar","generation_solar") \
    .withColumnRenamed("generation waste","generation_waste") \
    .withColumnRenamed("generation wind offshore","generation_wind_offshore") \
    .withColumnRenamed("generation wind onshore","generation_wind_onshore") \
    .withColumnRenamed("forecast solar day ahead","forecast_solar_day_ahead") \
    .withColumnRenamed("forecast wind offshore eday ahead","forecast_wind_offshore_day_ahead") \
    .withColumnRenamed("forecast wind onshore day ahead","forecast_wind_onshore_day_ahead") \
    .withColumnRenamed("total load forecast","total_load_forecast") \
    .withColumnRenamed("total load actual","total_load_actual") \
    .withColumnRenamed("price day ahead","price_day_ahead") \
    .withColumnRenamed("price actual","price_actual") 

In [3]:
# Check whether all mutations went correctly.
df.printSchema()
print(df.rdd.id())

df2.printSchema()
print(df2.rdd.id())

root
 |-- time: string (nullable = true)
 |-- generation biomass: string (nullable = true)
 |-- generation fossil brown coal/lignite: string (nullable = true)
 |-- generation fossil coal-derived gas: string (nullable = true)
 |-- generation fossil gas: string (nullable = true)
 |-- generation fossil hard coal: string (nullable = true)
 |-- generation fossil oil: string (nullable = true)
 |-- generation fossil oil shale: string (nullable = true)
 |-- generation fossil peat: string (nullable = true)
 |-- generation geothermal: string (nullable = true)
 |-- generation hydro pumped storage aggregated: string (nullable = true)
 |-- generation hydro pumped storage consumption: string (nullable = true)
 |-- generation hydro run-of-river and poundage: string (nullable = true)
 |-- generation hydro water reservoir: string (nullable = true)
 |-- generation marine: string (nullable = true)
 |-- generation nuclear: string (nullable = true)
 |-- generation other: string (nullable = true)
 |-- gener

In [38]:
# Transform df2 to a new variable where we take the types of the columns into account, since we saw above that the dataset stores all variables as String type.
import pyspark.sql.functions as F

df3 = df2.withColumn("generation_solar", df2["generation_solar"].cast('float')) \
    .withColumn("day", F.dayofweek(df2["ts"]).cast('integer')) \
    .withColumn("month", F.month(df2["ts"]).cast('integer')) \
    .withColumn("year", F.year(df2["ts"]).cast('integer')) \
    .withColumn("hour", F.hour(df2["ts"]).cast('integer')) \
    .withColumn("generation_wind_offshore", df2["generation_wind_offshore"].cast('float')) \
    .withColumn("generation_wind_onshore", df2["generation_wind_onshore"].cast('float')) \
    .withColumn("forecast_solar_day_ahead", df2["forecast_solar_day_ahead"].cast('float')) \
    .withColumn("forecast_wind_offshore_day_ahead", df2["forecast_wind_offshore_day_ahead"].cast('float')) \
    .withColumn("forecast_wind_onshore_day_ahead", df2["forecast_wind_onshore_day_ahead"].cast('float')) \
    .withColumn("price_day_ahead", df2["price_day_ahead"].cast('float')) \
    .withColumn("price_actual", df2["price_actual"].cast('float'))

In [52]:
#df3.head(25)

In [46]:
# Create a new dataframe df4 where we only store the columns relevant for the first data pipeline.
columns= ["ts","year", "month", "day", "hour", "generation_solar","generation_wind_offshore","generation_wind_onshore","forecast_solar_day_ahead"
         , "forecast_wind_onshore_day_ahead", "price_day_ahead", "price_actual"]
df4 = df3.select(columns)

In [47]:
# Check the effect of deleted columns
df5 = df4.dropna()
print((df4.count(), len(df4.columns)))
print((df5.count(), len(df5.columns)))

(35064, 12)
(35046, 12)


In [48]:
# Create the final sub dataframe df6 with the correct timestamp variable
df6 = df5.withColumn("generation_solar", df5["generation_solar"].cast('integer')) \
    .withColumn("generation_wind_offshore", df5["generation_wind_offshore"].cast('integer')) \
    .withColumn("generation_wind_onshore", df5["generation_wind_onshore"].cast('integer')) \
    .withColumn("forecast_solar_day_ahead", df5["forecast_solar_day_ahead"].cast('integer')) \
    .withColumn("forecast_wind_onshore_day_ahead", df5["forecast_wind_onshore_day_ahead"].cast('integer')) \
    .withColumn("price_day_ahead", df5["price_day_ahead"].cast('float')) \
    .withColumn("price_actual", df5["price_actual"].cast('float')) \
    .withColumn("ts", df5["ts"].cast('timestamp'))

In [49]:
df6.head(1)

[Row(ts=datetime.datetime(2014, 12, 31, 23, 0), year=2015, month=1, day=5, hour=23, generation_solar=49, generation_wind_offshore=0, generation_wind_onshore=6378, forecast_solar_day_ahead=17, forecast_wind_onshore_day_ahead=6436, price_day_ahead=50.099998474121094, price_actual=65.41000366210938)]

In [50]:
# Save df6 as a csv file in the jupyter lab environment of the virtual machine.
df6.toPandas().to_csv('energy_dataset_preprocessed.csv')

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


In [51]:
df6.head(1)

[Row(ts=datetime.datetime(2014, 12, 31, 23, 0), year=2015, month=1, day=5, hour=23, generation_solar=49, generation_wind_offshore=0, generation_wind_onshore=6378, forecast_solar_day_ahead=17, forecast_wind_onshore_day_ahead=6436, price_day_ahead=50.099998474121094, price_actual=65.41000366210938)]